In [1]:
import pandas as pd
import os
import numpy as np

# Importando tabelas e gerando listas

In [2]:
df_dados = pd.read_excel('dados.xlsx')
df_fator = pd.read_excel('fatores.xlsx')

lst_meses = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']
lst_meses_dados = ['Dez anterior','Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov']
lst_layout = ['Galpões', 'Complemento','Registro', 'Empresa', 'Tipo de imóvel']
lst_df_empresas = []


# Gerando Data Frame das diferenças

In [3]:
df_diferença = df_dados.loc[:, lst_layout]

for m, d in zip(lst_meses, lst_meses_dados):
    dif = df_dados.loc[:, m] - df_dados.loc[:, d]
    df_diferença.loc[:, m] = dif


df_diferença = df_diferença.loc[~df_diferença['Jan'].isna(), :].reset_index(drop=True)

df_fator.loc[3, 'Volume max de água m3'] = 999999


# Definição de funções

In [4]:
# criação de função DataFrame para adição de row TOTAL
def DF_total(df):
    num_index = df['Registro'].value_counts().sum()
    df.loc[num_index,:] = df.sum(numeric_only=True, axis=0)
    df.loc[num_index, 'Galpões'] = 'TOTAL'


# criação de função DataFrame para cada empresa. Obs: a variavel ainda será definida antes do uso
def DF_droper_empresa(df, comp):
    df_temp = df.loc[df['Empresa'] == comp, :].reset_index(drop=True)
    # df_temp = df.loc[df['Empresa'] == comp, ['Galpões','Complemento','Registro']+lst_meses_uteis].reset_index(drop=True)
    # df_temp = df_temp.dropna(how='all', axis=1)
    return df_temp


# criando função de adição de linhas vazias
def DF_empty_rows_comb(df):
    
    dc_empty = {col: [np.nan for _ in range(1)] for col in df.columns}
    df_empty = pd.DataFrame(dc_empty)
    df_new = pd.concat([df_empty, df, df_empty], axis=0, ignore_index=True)
    return df_new


# Retirando Colunas inutilizadas

In [5]:
# retirando colunas de meses não marcados para consumo de água
df_diferença = df_diferença.dropna(how='all', axis=1)


# retirando colunas de meses não marcados para leitura do hidrômetro
df_dados = df_dados.dropna(how='all', axis=1)


# lista com meses completos
lst_meses_uteis = df_diferença.columns.tolist()
for i in ['Galpões', 'Complemento', 'Registro', 'Empresa', 'Tipo de imóvel']:
    lst_meses_uteis.remove(i)


# Gerando Data Frame dos preços

In [6]:
# retirando informações de poço 
df_preço = df_diferença.loc[df_diferença['Galpões'] != 'Poço', lst_layout]


# calculando e multiplicando o fator para cada diferença. O loop m: meses ; loop type: tipo de imóvel ; loop index: para cada intervalo multiplicador
for m in lst_meses_uteis:
    
    for type in ['Industrial', 'Residencial']:
        df_preço.loc[(df_diferença[m] < df_fator.loc[0,'Volume max de água m3']) & (df_diferença['Tipo de imóvel'] == type), m] = df_fator.loc[0, f'Fator {type}']
        
        for index in range(1, 4):
            df_preço_temp = df_diferença.loc[(df_diferença[m] < df_fator.loc[index,'Volume max de água m3']) & (df_diferença['Tipo de imóvel'] == type) & (df_diferença[m] >= df_fator.loc[index, 'Volume min de água m3']), m]  
            df_preço.loc[(df_diferença[m] < df_fator.loc[index,'Volume max de água m3']) & (df_diferença['Tipo de imóvel'] == type) & (df_diferença[m] >= df_fator.loc[index, 'Volume min de água m3']), m] = df_preço_temp * df_fator.loc[index, f'Fator {type}']


# retirando colunas de meses não marcados
df_preço = df_preço.dropna(how='all', axis=1)

# TOTAL
DF_total(df_preço)
df_preço = df_preço.round(2)


# Formatação e geração de arquivo Tabela.xlsx

In [ ]:
# formatação R$ em um novo DataFrame
df_preço_formatado = df_preço.loc[:,:]
df_preço_formatado[lst_meses_uteis] = df_preço_formatado[lst_meses_uteis].map(lambda x: f'R$ {x}')

# formatação m3 em um novo DataFrame
df_diferença_formatado = df_diferença.loc[:,:]
df_diferença_formatado = df_diferença_formatado.round(2)
df_diferença_formatado[lst_meses_uteis] = df_diferença_formatado[lst_meses_uteis].map(lambda x: f'{x} m³')

# criação de arquivo excel
with pd.ExcelWriter("Tabela.xlsx", engine="openpyxl") as writer:
    df_preço_formatado.to_excel(writer, sheet_name="Custo água", index=False)

# criação de arquivo excel
with pd.ExcelWriter("Tabela.xlsx", engine="openpyxl", mode="a") as writer:
    df_diferença_formatado.to_excel(writer, sheet_name="Consumo água", index=False)
    

# Separando arquivos na pasta compartilhamento por empresa

In [7]:
if not os.path.exists(r'compartilhamento'):
    os.mkdir(r'compartilhamento')


# lista de empresas
lst_empresa = df_preço['Empresa'].value_counts().index.tolist()


for comp in lst_empresa:

    # criação de DataFrame do custo $ para cada empresa
    # criação de DataFrame do consumo m3 para cada empresa
    # criação de DataFrame do registro para cada empresa
    df_preço_temp = DF_droper_empresa(df_preço, comp)
    df_diferença_temp = DF_droper_empresa(df_diferença, comp)
    df_dados_temp = DF_droper_empresa(df_dados, comp)

    # implementação de TOTAL para custo $
    # implementação de TOTAL para consumo m3
    DF_total(df_preço_temp)
    DF_total(df_diferença_temp)
    
    
    # formatação de R$
    # formatação de m3
    df_preço_temp[lst_meses_uteis] = df_preço_temp[lst_meses_uteis].map(lambda x: f'R$ {x}')
    df_diferença_temp[lst_meses_uteis] = df_diferença_temp[lst_meses_uteis].map(lambda x: f'{x:.1f} m³')


    # empty rows
    df_dados_temp = DF_empty_rows_comb(df_dados_temp)
    df_diferença_temp = DF_empty_rows_comb(df_diferença_temp)
    df_preço_temp = DF_empty_rows_comb(df_preço_temp)

    # informações de cada tabela
    df_dados_temp.loc[0,'Galpões'] = 'LEITURA HIDRÔMETROS'
    df_preço_temp.loc[0,'Galpões'] = 'CUSTO ÁGUA'
    df_diferença_temp.loc[0,'Galpões'] = 'CONSUMO ÁGUA'
    
    
    # combinando DataFrames
    df_final_formatado_temp = pd.concat([df_dados_temp, df_diferença_temp, df_preço_temp], axis=0, ).reset_index(drop=True)

    

    # adicionando DataFrames para uma lista
    lst_df_empresas.append(df_final_formatado_temp)
    
    
    # geração de arquivo excel
    df_final_formatado_temp.to_excel(fr'compartilhamento\{comp}.xlsx', index=False)

    